# **IMAGE GENERATION LOOP**

In [1]:
import os
import json
import time
import importlib
from dotenv import load_dotenv
import functions
importlib.reload(functions)
from functions import (
    build_super_prompt, 
    generate_image_with_pollinations,
    grade_image_quality,
    init_gemini_client,
    get_latest_image
)

load_dotenv()
POLLINATION_API_KEY = os.environ.get('POLLINATION_API_KEY')
STRATEGY_FILE = 'current_strategy.json'
IMAGE_FOLDER = "pinterest_images"
MAX_RETRIES = 3

grader = init_gemini_client(temperature=0, response_mime_type="application/json")

if os.path.exists(STRATEGY_FILE):
    with open(STRATEGY_FILE, 'r') as f:
        strategy = json.load(f)
    
    base_prompt = build_super_prompt(strategy)
    
    img_path = get_latest_image(IMAGE_FOLDER)
    
    if not img_path:
        print("No existing image found. Generating initial image...")
        img_path = generate_image_with_pollinations(base_prompt, api_key=POLLINATION_API_KEY, folder=IMAGE_FOLDER)
    else:
        print(f"Found existing image: {img_path}")

    for i in range(MAX_RETRIES + 1):
        if not img_path:
            print("Generation failed at start of loop.")
            break

        print(f"\nGrading Attempt {i} (Image: {os.path.basename(img_path)})...")
        score_data = grade_image_quality(img_path, grader)
        score = score_data.get('overall_score', 0)
        print(f"Score: {score}/10 | Tip: {score_data.get('tip')}")
        
        if score > 5:
            print("Image Accepted!")
            break
        
        if i < MAX_RETRIES:
            print(f"Score too low (<=5). Retrying with variation {i+1}...")
            variation_prompt = f"{base_prompt} ultra high quality, variation {i+1}"
            img_path = generate_image_with_pollinations(variation_prompt, api_key=POLLINATION_API_KEY, folder=IMAGE_FOLDER)
        else:
            print("Max retries reached. Keeping last image.")
            
else:
    print(f" {STRATEGY_FILE} not found. Run brain.ipynb first.")

Found existing image: pinterest_images\pin_1770254396.jpg

Grading Attempt 0 (Image: pin_1770254396.jpg)...
Score: 9/10 | Tip: To further enhance scroll-stopping potential, consider adding a subtle, unique decorative item or a slightly more vibrant accent color.
Image Accepted!
